In [29]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.autograd import Variable

# Define the custom dataset class
class PaddleDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data.iloc[idx, :8], dtype=torch.float32)
        label = torch.tensor(self.data.iloc[idx, 8:13].values, dtype=torch.float32)
        return features, label

# Define the neural network model
class PaddleClassifier(nn.Module):
    def __init__(self):
        super(PaddleClassifier, self).__init__()
        self.fc1 = nn.Linear(8, 32)
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [30]:
# Load the data
data = pd.read_csv("TrainingDataReorg.tsv", sep="\t")
train_dataset = PaddleDataset(data)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(train_dataset[12])

(tensor([-0.0800,  0.5166, -0.1400,  3.0000,  0.6613, -0.2327,  1.0667,  3.1500]), tensor([0., 0., 1., 0., 0.]))


In [32]:
# Initialize the model, loss function, and optimizer
model = PaddleClassifier().to(device)
criterion = nn.MSELoss();
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model_path = "paddle_classifier.pth"
onnx_path = "paddle_classifier.onnx"

# Load the model from disk if it exists
if os.path.isfile(model_path):
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    print("Model loaded from disk")

In [33]:
# Train the model
epochs = 30
for epoch in range(epochs):
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # if (epoch + 1) % 10 == 0:
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

print("Training complete")

Epoch [1/30], Loss: 0.1363
Epoch [2/30], Loss: 0.0517
Epoch [3/30], Loss: 0.0916
Epoch [4/30], Loss: 0.0644
Epoch [5/30], Loss: 0.0171
Epoch [6/30], Loss: 0.0283
Epoch [7/30], Loss: 0.0135
Epoch [8/30], Loss: 0.0978
Epoch [9/30], Loss: 0.0273
Epoch [10/30], Loss: 0.0525
Epoch [11/30], Loss: 0.0420
Epoch [12/30], Loss: 0.0915
Epoch [13/30], Loss: 0.0765
Epoch [14/30], Loss: 0.1017
Epoch [15/30], Loss: 0.1963
Epoch [16/30], Loss: 0.1398
Epoch [17/30], Loss: 0.0437
Epoch [18/30], Loss: 0.1648
Epoch [19/30], Loss: 0.0592
Epoch [20/30], Loss: 0.0772
Epoch [21/30], Loss: 0.0859
Epoch [22/30], Loss: 0.0987
Epoch [23/30], Loss: 0.1685
Epoch [24/30], Loss: 0.0094
Epoch [25/30], Loss: 0.0761
Epoch [26/30], Loss: 0.1467
Epoch [27/30], Loss: 0.0696
Epoch [28/30], Loss: 0.1201
Epoch [29/30], Loss: 0.0180
Epoch [30/30], Loss: 0.0112
Training complete


In [34]:
# Save the trained model to disk
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Move the model back to CPU before exporting to ONNX
model.to("cpu")

# Export the model to ONNX format
dummy_input = Variable(torch.randn(1, 8))
torch.onnx.export(model, dummy_input, onnx_path, verbose=True, input_names=["input"], output_names=["output"])

# Check the ONNX model
#onnx_model = torch.onnx.load(onnx_path)
#onnx.checker.check_model(onnx_model)
print(f"ONNX model saved to {onnx_path}")

Model saved to paddle_classifier.pth
ONNX model saved to paddle_classifier.onnx


In [35]:
#test model with set inputs
import numpy as np

def predict(model, input_data):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return probabilities, predicted_class

# Example input data
input_data = [2.02, 28.22204, 0.4549797, 0, -0.377058, -0.2713, -0.4834, -3.15]  # Replace this with your own input data
probabilities, predicted_class = predict(model, input_data)

# Map the predicted_class index to the corresponding label
class_labels = ['MyPaddleIsRight', 'MyPaddleIsLeft', 'MyPaddleIsStill']
predicted_label = class_labels[predicted_class]

print(f"Probabilities: {probabilities.cpu().numpy()}")
print(f"Predicted class index: {predicted_class}")
print(f"Predicted class label: {predicted_label}")


Probabilities: [0.01453722 0.00663264 0.9607708  0.0099114  0.00814788]
Predicted class index: 2
Predicted class label: MyPaddleIsStill
